In [1]:
from convlstm import SeismicConvLSTM
from customLoss import CustomLoss
import torch
from dataloader import train_dataloader, val_dataloader, test_dataloader
from evaluater import evaluate_model


---------------- Dataset information: ----------------
[I] Sample rate: 100 Hz
[I] Chosen window size:900
[I] Hopping size:300
[I] Each sample is window_size/sample rate seconds long. Each sample has 3 channels.
[I] Dataset values shape is: [num_samples, num_window, num_channel, (height) 1, (width) num_sample_points]
[I] Dataset value shape: torch.Size([1575, 3, 3, 1, 900])
[I] Dataset labels shape is: [num_samples, 4], where the 4 values are [p_idx, s_idx, no_PS, only_P, only_S, both_PS]
[I] Dataset label shape:torch.Size([4725, 6])
[I] One sample shape: torch.Size([3, 3, 1, 900])
[I] One label shape: torch.Size([6])
[I] ----------------------------------------------------
[I] Some label examples in the dataset:
[I] Label: tensor([-1., -1.,  1.,  0.,  0.,  0.])
[I] Label: tensor([894.,  -1.,   0.,   1.,   0.,   0.])
[I] Label: tensor([359., 953.,   0.,   0.,   0.,   1.])
[I] Label: tensor([    1286.,       -1.,        0.,        1.,        0.,        0.])
[S] -------------------------

In [5]:
# Define the model
# We are dynamically fetching the input channels, num_windows, data_points_of_each_window, and the output_dim from the dataset using the dataloader
one_sample = next(iter(train_dataloader)) # one_sample[0].shape, one_sample[1].shape --> torch.Size([16, 3, 3, 1, 900]) torch.Size([16, 4])
INPUT_CHANNELS = one_sample[0].shape[2] # one_sample[0].shape[2] --> 3
OUTPUT_DIM = one_sample[1].shape[1] # one_sample[1].shape[1] --> 4
NUM_WINDOWS = one_sample[0].shape[1] # one_sample[0].shape[1] --> 3
DATA_POINTS_OF_EACH_WINDOW = one_sample[0].shape[4] # one_sample[0].shape[4] --> 900
HIDDEN_DIM = [32, 32, 64, 64, 128]
KERNEL_SIZE = (11, 5, 3, 3, 3)    
NUM_CONVLSTM_BLOCKS = 5
model = SeismicConvLSTM(input_dim=INPUT_CHANNELS, hidden_dim=HIDDEN_DIM, kernel_size=KERNEL_SIZE, num_layers=NUM_CONVLSTM_BLOCKS, output_dim=OUTPUT_DIM, num_windows = NUM_WINDOWS, data_points = DATA_POINTS_OF_EACH_WINDOW)
# load the model from the saved file
model_name = "model_20241008-2306_epoch30_lr0.01_lambda2_windows3_datapoints900.pth"
model.load_state_dict(torch.load('models/'+model_name))

Layer 1
Layer 2


RuntimeError: Error(s) in loading state_dict for SeismicConvLSTM:
	size mismatch for convlstm.cell_list.0.conv.weight: copying a param with shape torch.Size([256, 67, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 67, 11, 1]).
	size mismatch for convlstm.cell_list.1.conv.weight: copying a param with shape torch.Size([256, 128, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 128, 5, 1]).

In [ ]:
# Evaluate the model with the test dataloader
criterion = CustomLoss(lambda_val=2)
train_losses = []
val_losses = []
output_path  = 'output/'+model_name.split('.')[0]+'_test'
evaluate_model(model, test_dataloader, criterion, output_path)

In [51]:
def collect_time_predictions(data_loader, model, device):
    """ Collect true and predicted P and S wave times from a DataLoader using the given model. """
    model.eval()
    print("Evaluation startedd...")
    true_times = []
    pred_times = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            
            outputs = model(inputs)
            p_orig, s_orig, no_ps_orig, only_p_orig, only_s_orig, both_ps_orig = labels[0]
            p_pred, s_pred, no_ps, only_p, only_s, both_ps = outputs[0]
        
            true_times.append((p_orig, s_orig))
            pred_times.append((p_pred, s_pred))
    print("Evaluation completed.")
    # true_times, pred_times = [], []
    # with torch.no_grad():
    #     for samples, p_times, s_times, _ in data_loader:
    #         samples = samples.to(device)
    #         p_pred, s_pred, _ = model(samples)
    #         true_times.append((p_times.cpu(), s_times.cpu()))
    #         pred_times.append((p_pred.cpu(), s_pred.cpu()))
    return true_times, pred_times

def flatten_and_convert(data_list):
    """ Flatten each array in the list and convert tensors to numpy arrays if needed. """
    return [data.numpy().flatten() if isinstance(data, torch.Tensor) else np.array(data).flatten()
            for data_tuple in data_list for data in data_tuple]


In [ ]:

# # Collect data for train, validation, and test sets
train_true_times, train_pred_times = collect_time_predictions(train_dataloader, model, device='cpu')
val_true_times, val_pred_times = collect_time_predictions(val_dataloader, model, device='cpu')
test_true_times, test_pred_times = collect_time_predictions(test_dataloader, model, device='cpu')


In [ ]:

# # Unpack and flatten the true and predicted times for P and S waves
# p_times_true_train_flat = flatten_and_convert([times[0] for times in train_true_times])
# s_times_true_train_flat = flatten_and_convert([times[1] for times in train_true_times])
# p_times_pred_train_flat = flatten_and_convert([times[0] for times in train_pred_times])
# s_times_pred_train_flat = flatten_and_convert([times[1] for times in train_pred_times])

# p_times_true_val_flat = []
# s_times_true_val_flat = []
# p_times_pred_val_flat = []
# s_times_pred_val_flat = []

# p_times_true_test_flat = []
# s_times_true_test_flat = []
# p_times_pred_test_flat = []
# s_times_pred_test_flat = []

# for i in val_true_times:
#     p_times_true_val_flat.append(i[0].numpy().flatten())
#     s_times_true_val_flat.append(i[1].numpy().flatten())
# for i in val_pred_times:
#     p_times_pred_val_flat.append(i[0].numpy().flatten())
#     s_times_pred_val_flat.append(i[1].numpy().flatten())   
# for i in test_true_times:
#     p_times_true_test_flat.append(i[0].numpy().flatten())
#     s_times_true_test_flat.append(i[1].numpy().flatten())
# for i in test_pred_times:
#     p_times_pred_test_flat.append(i[0].numpy().flatten())
#     s_times_pred_test_flat.append(i[1].numpy().flatten())

def flatten_and_convert(data_list):
    """ Flatten each array in the list and convert tensors to numpy arrays if needed. """
    return [data.item() if isinstance(data, torch.Tensor) else data.flatten().item()
            for data_tuple in data_list for data in data_tuple]

p_times_true_train_flat = flatten_and_convert([(times[0],) for times in train_true_times])
s_times_true_train_flat = flatten_and_convert([(times[1],) for times in train_true_times])
p_times_pred_train_flat = flatten_and_convert([(times[0],) for times in train_pred_times])
s_times_pred_train_flat = flatten_and_convert([(times[1],) for times in train_pred_times])

p_times_true_val_flat = flatten_and_convert([(times[0],) for times in val_true_times])
s_times_true_val_flat = flatten_and_convert([(times[1],) for times in val_true_times])
p_times_pred_val_flat = flatten_and_convert([(times[0],) for times in val_pred_times])
s_times_pred_val_flat = flatten_and_convert([(times[1],) for times in val_pred_times])

p_times_true_test_flat = flatten_and_convert([(times[0],) for times in test_true_times])
s_times_true_test_flat = flatten_and_convert([(times[1],) for times in test_true_times])
p_times_pred_test_flat = flatten_and_convert([(times[0],) for times in test_pred_times])
s_times_pred_test_flat = flatten_and_convert([(times[1],) for times in test_pred_times])

print(p_times_true_val_flat[0], p_times_true_val_flat[1], p_times_true_val_flat[2], p_times_true_val_flat[3])

import matplotlib.pyplot as plt
import numpy as np



# Dividing all values in each array by 100
p_times_true_train_flat_n = np.array(p_times_true_train_flat) / 100
p_times_pred_train_flat_n = np.array(p_times_pred_train_flat) / 100
s_times_true_train_flat_n = np.array(s_times_true_train_flat) / 100
s_times_pred_train_flat_n = np.array(s_times_pred_train_flat) / 100

p_times_true_val_flat_n = np.array(p_times_true_val_flat) / 100
p_times_pred_val_flat_n = np.array(p_times_pred_val_flat) / 100
s_times_true_val_flat_n = np.array(s_times_true_val_flat) / 100
s_times_pred_val_flat_n = np.array(s_times_pred_val_flat) / 100

p_times_true_test_flat_n = np.array(p_times_true_test_flat) / 100
p_times_pred_test_flat_n = np.array(p_times_pred_test_flat) / 100
s_times_true_test_flat_n = np.array(s_times_true_test_flat) / 100
s_times_pred_test_flat_n = np.array(s_times_pred_test_flat) / 100





In [54]:
def plot_combined_true_vs_predicted_with_deviation(true_data, pred_data, titles, wave_type):
    """ Plot True vs. Predicted times for P or S waves across Train, Validation, and Test datasets. """
    colors = ['red', 'green', 'blue']  # Colors for Train, Validation, Test
    labels = ['Train', 'Validation', 'Test']

    plt.figure(figsize=(10, 8))
    for true, pred, color, label in zip(true_data, pred_data, colors, labels):
        true = np.array(true)
        pred = np.array(pred)

        # Plotting actual vs predicted
        plt.scatter(true, pred, color=color, label=f'{label} Actual vs. Predicted', alpha=0.6)
        plt.plot([true.min(), true.max()], [true.min(), true.max()], 'k--', lw=2)  # Line of perfect prediction
        #plt.plot(true, true * 1.3, '--', color=color, lw=1)  # +30% deviation line
        #plt.plot(true, true * 0.7, '--', color=color, lw=1)  # -30% deviation line
        plt.plot(true, true + 3.0, '--', color=color, lw=1)  # +3 second deviation line
        plt.plot(true, true - 3.0, '--', color=color, lw=1)  # -3 second deviation line


    # Define the maximum range for ticks based on the data
    max_time = max([t.max() for t in true] + [p.max() for p in pred])
    major_ticks = np.arange(0, np.ceil(max_time) + 1, 1)  # Major ticks every 1 second
    #minor_ticks = np.arange(0, np.ceil(max_time) + 0.1, 0.2)  # Minor ticks every 0.2 seconds

    plt.xticks(major_ticks)
    plt.yticks(major_ticks)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    #plt.minorticks_on()
    #plt.grid(True, which='minor', linestyle='-', color='grey', linewidth=0.5)  # Thin grey lines for minor ticks


    plt.xlabel('Actual Time (s)')
    plt.ylabel('Predicted Time (s)')
    #plt.title(f'True vs. Predicted {wave_type} Times with 30% Deviation')
    plt.title(f'True vs. Predicted {wave_type} Times with -+3 Seconds Deviation')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
plot_combined_true_vs_predicted_with_deviation(
    [p_times_true_train_flat_n, p_times_true_val_flat_n, p_times_true_test_flat_n],
    [ p_times_pred_train_flat_n, p_times_pred_val_flat_n, p_times_pred_test_flat_n],
    [ 'Train','Validation', 'Test'],
    'P'
)

plot_combined_true_vs_predicted_with_deviation(
    [s_times_true_train_flat_n, s_times_true_val_flat_n, s_times_true_test_flat_n],
    [s_times_pred_train_flat_n, s_times_pred_val_flat_n, s_times_pred_test_flat_n],
    [ 'Train','Validation', 'Test'],
    'S'
)

